### Load Libraries

In [2]:
# Pandas for dataframes
import pandas as pd

# Import Team Class
from Classes.team import Team

# Pickle import for data serialization and deserialization
import pickle

# csv files from reading and writing
import csv

# Nan for dictionary creation
import math

### Load Data & Define Constants

In [3]:
# Read Pickle File as dictionary
teams_dictionary_file = open("Data/teams_dictionary.pkl", "rb")
teams_dictionary = pickle.load(teams_dictionary_file)

# Load in stat labels csv as dataframe (then convert to list)
all_labels_df = pd.read_csv("Data/stat_labels.csv")
stat_labels_list = list(all_labels_df['Stat Label'])

# Load in keys (team names) csv as dataframe
results_csv = pd.read_csv('Data/Results/madness_results_confirmed.csv') 

target_years = ['2012-13','2013-14','2014-15','2015-16','2016-17','2017-18','2018-19','2019-20','2020-21','2021-22']

### Update Teams Dictionary

- Input: Target Data Years (reads csv files in Data/Yearly folder)
- Output: Writes teams_dictionary to pkl file

In [5]:
def Update_Teams_Dict(years):

    for year in target_years:
        file_to_read = open(f"Data/Yearly/{year}_data_dictionary.pkl", "rb")
        yearly_data = pickle.load(file_to_read)

        for team in yearly_data.keys():
            teams_dictionary[team].stat_year_data[year] = yearly_data[team][year]

    # create a binary pickle file for main dataframe export 
    f = open(f"Data/teams_dictionary.pkl","wb")

    # write the python object (dict) to pickle file
    pickle.dump(teams_dictionary,f)

    # close file
    f.close()

##### Retrieve Results Data into Dictionary

- Input: Results csv file: 'Data/Results/madness_results_confirmed.csv'
- Output: Results data dictionary
- Writes dictionary to pkl file: result_dictionary.pkl

In [45]:
def retrieve_madness_results(results):

    # Data Dictionary for all results
    results_data_dictionary = {}

    year_array = results.columns[1:]

    for team in range(len(results['Team Name'])):

        team_name = results['Team Name'][team]

        rankings_dictionary = {}

        for year in year_array:
            ranking = results[year][team]
            rankings_dictionary[year] = ranking

        results_data_dictionary[team_name] = rankings_dictionary

    f = open(f"Data/Results/results_dictionary.pkl","wb")

    # write the python object (dict) to pickle file
    pickle.dump(results_data_dictionary,f)

    # close file
    f.close()
    
    return results_data_dictionary

##### Compile Madness dataframe

- Input: Target Year, Data Labels, Results Dictionary
- Returns: Dataframe (formatted for analysis)
- Writes dataframe to csv: "Data/Madness/{year}.csv"

In [46]:
def df_year_compiler(year, data_labels, results_dictionary):

    # Read Pickle File as dictionary
    file_to_read = open(f"Data/teams_dictionary.pkl", "rb")
    teams_dict = pickle.load(file_to_read)

    madness_teams = []

    for key in results_dictionary:
        result = results_dictionary[key][year]
        if not math.isnan(result):
            madness_teams.append(key)
    
    year_df = pd.DataFrame(columns=data_labels)

    for team in madness_teams:

        if year in teams_dict[team].stat_years:
            team_stats = teams_dict[team].stat_year_data[year]
        else:
            continue

        #Final data format for dataframe upload
        team_name = teams_dict[team].name
        team_id = teams_dict[team].team_id
        madness_result = results_dictionary[team][year]

        team_info = [team_name, team_id, year, madness_result]

        all_data = team_info + team_stats

        if len(all_data) == len(data_labels):
            year_df.loc[len(year_df.index)] = all_data

    year_df.to_csv(f"Data/Madness/{year}.csv", index=False)
    return(year_df)

### Code Begins

In [47]:
Update_Teams_Dict(target_years)

In [48]:
results_dict = retrieve_madness_results(results=results_csv)

In [49]:
target_year = target_years[9]
team_data_labels = ['Team Name', 'Team ID', 'Test Year', 'Madness Result'] + stat_labels_list

In [50]:
df_year_compiler(year=target_year, data_labels=team_data_labels, results_dictionary=results_dict)

,Team Name,Team ID,Test Year,Madness Result,g,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,...,opp_ft_pct_rank,opp_orb_per_g_rank,opp_drb_per_g_rank,opp_trb_per_g_rank,opp_ast_per_g_rank,opp_stl_per_g_rank,opp_blk_per_g_rank,opp_tov_per_g_rank,opp_pf_per_g_rank,opp_pts_per_g_rank
0,Michigan Wolverines,236,2021-22,4.0,34,40.0,26.9,57.6,.467,20.6,...,282,48,23,12,263,46,204,13,107,188
1,Duke Blue Devils,110,2021-22,6.0,39,40.1,29.7,60.4,.491,21.8,...,59,312,43,143,348,119,199,41,91,134
2,Marquette Golden Eagles,220,2021-22,2.0,32,40.3,26.9,59.8,.450,18.3,...,107,353,315,350,336,270,193,257,60,251
3,Ohio State Buckeyes,298,2021-22,3.0,32,40.3,25.6,54.4,.471,17.8,...,270,247,10,50,152,144,38,17,230,132
4,Indiana Hoosiers,173,2021-22,2.0,35,40.4,25.8,56.8,.454,19.9,...,341,115,149,119,285,210,73,122,235,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Vermont Catamounts,437,2021-22,2.0,34,40.1,27.5,55.9,.492,18.8,...,63,3,42,3,315,9,97,55,126,12
64,Virginia Tech Hokies,442,2021-22,2.0,36,40.1,25.9,55.0,.471,17.1,...,316,66,19,14,269,167,150,111,47,24
65,Wright State Raiders,480,2021-22,2.0,36,40.0,27.4,59.4,.461,20.9,...,176,237,77,124,256,310,130,204,277,253
66,Wyoming Cowboys,481,2021-22,1.0,34,40.6,25.1,54.9,.457,17.0,...,221,86,119,83,50,25,65,38,336,62


Compile combined dataframe

In [10]:
all_years_df = pd.DataFrame(columns=stat_labels_list)

for year in target_years:
    year_df = pd.read_csv(f"Data/Madness/{year}.csv")
    all_years_df = pd.concat([all_years_df, year_df])


all_years_df.to_csv(f"Data/Madness/all_years.csv", index=False)

